In [1]:
import pandas as pd
import numpy as np 
import keras 
from keras.layers import Dense, Dropout

C:\Users\Szymon\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Szymon\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Szymon\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Szymon\Anaconda3\lib\site-packages\tensorflow\python\framew

In [130]:
from keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D

In [80]:
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

In [2]:
from statistics import mean

In [4]:
from sklearn.model_selection import train_test_split

In [211]:
headers = ['sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'decision']
data = pd.read_csv('output2.csv', names=headers)
data.head()

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,decision
0,398.0,101.857902,60.925663,474.156302,231.819254,straight
1,396.0,101.542193,60.925663,472.084688,231.819254,straight
2,394.0,101.542193,60.925663,470.015662,231.819254,straight
3,392.0,101.542193,60.925663,467.944048,231.819254,straight
4,390.0,101.542193,60.925663,465.875022,231.819254,straight


In [212]:
df = pd.get_dummies(data, columns=['decision'])

In [213]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8002 entries, 0 to 8001
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sensor_1           8002 non-null   float64
 1   sensor_2           8002 non-null   float64
 2   sensor_3           8002 non-null   float64
 3   sensor_4           8002 non-null   float64
 4   sensor_5           8002 non-null   float64
 5   decision_left      8002 non-null   uint8  
 6   decision_right     8002 non-null   uint8  
 7   decision_straight  8002 non-null   uint8  
dtypes: float64(5), uint8(3)
memory usage: 336.1 KB


In [214]:
x = df[list(df.columns)[:5]]
y = df[list(df.columns)[5:]]

In [215]:
X_train, X_test, y_train, y_test = train_test_split(x, y, shuffle=True, test_size=0.33, random_state=42, stratify=data['decision'])

In [216]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

X_train = np.clip(X_train, -5, 5)
X_test = np.clip(X_test, -5, 5)

In [217]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [218]:
class_weights={0: 1 - sum([el[0] for el in y_train]) / len(y_train), # left
               1: 1 - sum([el[1] for el in y_train]) / len(y_train), # right
               2: 1 - sum([el[2] for el in y_train]) / len(y_train), # straight
              }


In [219]:
class_weights

{0: 0.7338183174780825, 1: 0.9350867375489648, 2: 0.3310949449729528}

# Model creation


In [220]:
def build_model():
    network = keras.Sequential()
    network.add(Dense(4000, activation='relu', input_shape=(5,)))
    network.add(Dense(3, activation='softmax'))
    network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    return network

In [221]:
from keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

def build_model_2():
    K.clear_session()
    inputs = Input(shape=(5,1))
    #First Conv1D layer
    conv = Conv1D(45,3, padding='valid', activation='relu', strides=1)(inputs)
    conv = MaxPooling1D(3)(conv)
    conv = Dropout(0.3)(conv)
    
    #Flatten layer
    conv = Flatten()(conv)

    #Dense Layer 1
    conv = Dense(4500, activation='relu')(conv)
    conv = Dropout(0.3)(conv)
    outputs = Dense(3, activation='softmax')(conv)

    model = Model(inputs, outputs)
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [222]:
def evaluate_classifier(model_f, train_data, train_targets, epochs, class_weights):
    k = 5
    num_val_samples = len(train_data) // k
    num_epochs = epochs
    all_scores = []
    for i in range(k):
        print('processing fold #', i)
        # Prepare the validation data: data from partition # k
        val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
        val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

        # Prepare the training data: data from all other partitions
        partial_train_data = np.concatenate(
            [train_data[:i * num_val_samples],
             train_data[(i + 1) * num_val_samples:]],
            axis=0)
        partial_train_targets = np.concatenate(
            [train_targets[:i * num_val_samples],
             train_targets[(i + 1) * num_val_samples:]],
            axis=0)

        # Build the Keras model (already compiled)
        model = model_f()
        # Train the model (in silent mode, verbose=0)
        model.fit(partial_train_data, partial_train_targets,
                  epochs=num_epochs, batch_size=124, verbose=1, class_weight=class_weights)
        # Evaluate the model on the validation data
        val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1)
        all_scores.append(val_mae)
    return mean(all_scores)

In [224]:
evaluate_classifier(build_model, X_train, y_train, 45, class_weights)

processing fold # 0
Epoch 1/45
4289/4289 [==============================] - 0s 77us/step - loss: 0.3784 - accuracy: 0.7554
Epoch 2/45
4289/4289 [==============================] - 0s 21us/step - loss: 0.3548 - accuracy: 0.7696
Epoch 3/45
4289/4289 [==============================] - 0s 21us/step - loss: 0.3440 - accuracy: 0.7668
Epoch 4/45
4289/4289 [==============================] - 0s 22us/step - loss: 0.3396 - accuracy: 0.7603
Epoch 5/45
4289/4289 [==============================] - 0s 20us/step - loss: 0.3332 - accuracy: 0.7550
Epoch 6/45
4289/4289 [==============================] - 0s 21us/step - loss: 0.3298 - accuracy: 0.7603
Epoch 7/45
4289/4289 [==============================] - 0s 21us/step - loss: 0.3289 - accuracy: 0.7470
Epoch 8/45
4289/4289 [==============================] - 0s 22us/step - loss: 0.3257 - accuracy: 0.7585
Epoch 9/45
4289/4289 [==============================] - 0s 22us/step - loss: 0.3227 - accuracy: 0.7568
Epoch 10/45
4289/4289 [==============================

4289/4289 [==============================] - 0s 21us/step - loss: 0.3036 - accuracy: 0.7589
Epoch 34/45
4289/4289 [==============================] - 0s 21us/step - loss: 0.3042 - accuracy: 0.7633
Epoch 35/45
4289/4289 [==============================] - 0s 21us/step - loss: 0.3033 - accuracy: 0.7626
Epoch 36/45
4289/4289 [==============================] - 0s 21us/step - loss: 0.3032 - accuracy: 0.7599
Epoch 37/45
4289/4289 [==============================] - 0s 20us/step - loss: 0.3013 - accuracy: 0.7636
Epoch 38/45
4289/4289 [==============================] - 0s 20us/step - loss: 0.3045 - accuracy: 0.7629
Epoch 39/45
4289/4289 [==============================] - 0s 20us/step - loss: 0.3012 - accuracy: 0.7640
Epoch 40/45
4289/4289 [==============================] - 0s 21us/step - loss: 0.3025 - accuracy: 0.7505
Epoch 41/45
4289/4289 [==============================] - 0s 21us/step - loss: 0.2998 - accuracy: 0.7592
Epoch 42/45
4289/4289 [==============================] - 0s 21us/step - loss

KeyboardInterrupt: 

In [225]:
model = build_model()

In [226]:
model.fit(X_train, y_train, epochs=40, validation_data=(X_test, y_test))

Train on 5361 samples, validate on 2641 samples
Epoch 1/40
5361/5361 [==============================] - 1s 143us/step - loss: 0.6086 - accuracy: 0.7752 - val_loss: 0.5664 - val_accuracy: 0.7823
Epoch 2/40
5361/5361 [==============================] - 1s 95us/step - loss: 0.5705 - accuracy: 0.7782 - val_loss: 0.5426 - val_accuracy: 0.7861
Epoch 3/40
5361/5361 [==============================] - 0s 92us/step - loss: 0.5575 - accuracy: 0.7840 - val_loss: 0.5380 - val_accuracy: 0.7876
Epoch 4/40
5361/5361 [==============================] - 1s 101us/step - loss: 0.5507 - accuracy: 0.7819 - val_loss: 0.5377 - val_accuracy: 0.7827
Epoch 5/40
5361/5361 [==============================] - 1s 108us/step - loss: 0.5472 - accuracy: 0.7791 - val_loss: 0.5358 - val_accuracy: 0.7872
Epoch 6/40
5361/5361 [==============================] - 1s 106us/step - loss: 0.5436 - accuracy: 0.7840 - val_loss: 0.5282 - val_accuracy: 0.7827
Epoch 7/40
5361/5361 [==============================] - 1s 100us/step - loss: 

In [227]:
predictions = model.predict(X_test)

In [228]:
pred_labels = [np.argmax(el) for el in predictions]
true_labels = [np.argmax(el) for el in y_test]
cm = confusion_matrix(true_labels, pred_labels)
print("Confusion matrix:\n{}".\
      format(cm))

Confusion matrix:
[[ 335    5  363]
 [  30   35  106]
 [  47    1 1719]]


In [229]:
model.save('trained_model_test.h5')

In [230]:
from pickle import dump

In [231]:
dump(scaler, open('scaler.pkl', 'wb'))